## Product Q&A Chatbot using Python and LangChain
```
pip install -qU langchain-community faiss-cpu
```

1. CSV File: The program should read a CSV file (products.csv) with the following columns:
- title – name of the product
- description – details about the product
- price – cost of the product
2. Document Creation: Convert each row of the CSV into a Document that contains all product information.
3. Embeddings & Vector Store: Generate embeddings for each document and store them in a vector database (e.g., FAISS).
4. Question Answering: Implement a RetrievalQA chain using LangChain to:
- Retrieve relevant products based on the user’s question.
- Generate a clear and concise answer.
5. Interactive Chat: The program should run in a loop, letting users ask multiple questions until they type "exit".

In [ ]:
import os
import pandas as pd
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.chains import RetrievalQA
from langchain_core.documents import Document
from dotenv import load_dotenv

In [ ]:
load_dotenv()
aval_api_key=os.getenv("AVALAI_API_KEY")

In [ ]:
# --- Step 1: Load your CSV file ---
csv_file = "inputs/products.csv"
df = pd.read_csv(csv_file)

In [ ]:
# --- Step 2: Convert each row into a Document ---
documents = []
for _, row in df.iterrows():
    content = f"Title: {row['title']}\nDescription: {row['description']}\nPrice: {row['price']}"
    documents.append(Document(page_content=content))


In [ ]:
# --- Step 3: Create embeddings and vector store ---
embeddings = OpenAIEmbeddings(
    api_key=aval_api_key,
    base_url="https://api.avalai.ir/v1",
    model="text-embedding-3-small",
)

vectorstore = FAISS.from_documents(documents, embeddings)


In [ ]:
# --- Step 4: Create a RetrievalQA chain ---
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":1})
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(
        api_key=aval_api_key,
        base_url="https://api.avalai.ir/v1",
        model="gpt-4o-mini",
        temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# --- Step 5: Ask questions ---
while True:
    query = input("\nAsk a question about our products (or type 'exit'): ")
    if query.lower() == "exit":
        break
    result = qa(query)
    print("\nAnswer:\n", result['result'])
    print("\nSources:\n", [doc.page_content for doc in result['source_documents']])


## Execise :)
Add memory to this program